In [28]:
from aocd import get_data, submit
from util import *
from dataclasses import dataclass
%load_ext ipython_unittest

In [2]:
data = get_data(day=23)

In [67]:
class Node():
    def __init__(self, value: int):
        self.value = value
        self.cw = self
        self.ccw = self

    def insert_after(self, node):
        """ Inserts the chain started at node after self """
        # assert self.find_value(node.value) == None, "Cant double up values, mistakes were made"
        
        old_cw = self.cw
        self.cw = node
        old_cw.ccw = node.ccw
        node.ccw.cw = old_cw
        node.ccw = self
        
    def remove_next_n(self, n: int) -> 'Node':
        assert n > 0
        start = self.cw # first node to be removed
        end = start # last node to be removed
        # identify last node in to be removed, inclusive
        for i in range(n - 1):
            end = end.cw
        
        self.cw = end.cw # set next of original to whatever was after the end
        end.cw.ccw = self # complete the back direction
            
        start.ccw = end # and make the removed sequence a loop
        end.cw = start
        
        return start
    
    def find_next_lowest_value(self, key_map, ignore_values, max_value = 9) -> 'Node':
        mod = (max_value + 1)
        target = (self.value - 1) % mod
        while True:
            node = key_map.get(target) if target not in ignore_values else None
            if node:
                return node
            else:
                target = (target - 1) % mod
        
    def find_value(self, value) -> Optional['Node']:
        current = self
        while True:
            if current.value == value:
                return current
            current = current.cw
            if current == self:
                # we looped and never found the value
                return None
            
        
    def to_string(self):
        output = [self.value]
        current = self.cw
        while current != self:
            output.append(current.value)
            current = current.cw
        return ", ".join(mapl(str, output))
    
    def take(self, n) -> List[int]:
        output = []
        current = self
        for i in range(n):
            output.append(current.value)
            current = current.cw
        return output
    
    def __repr__(self):
        return f"{self.ccw.value} <- {self.value} -> {self.cw.value}"


In [68]:
%%unittest_testcase
def test_insert_1(self):
    current = Node(1)
    target = Node(2)
    current.insert_after(target)
    self.assertEqual(current.cw, target)
    self.assertEqual(current.ccw, target)
    self.assertEqual(target.cw, current)
    self.assertEqual(target.ccw, current)
    
def test_insert_3(self):
    current = Node(1)
    old_next = Node(4)
    current.insert_after(old_next)
    
    start = Node(1)
    start.insert_after(Node(2))
    end = Node(3)
    start.cw.insert_after(end)
    
    current.insert_after(start)
    self.assertEqual(current.cw, start)
    self.assertEqual(current.ccw, old_next)
    self.assertEqual(end.cw, old_next)
    self.assertEqual(start.ccw, current)
    
    start.take(4) == list(range(1,5))

Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [69]:
def parse_data(data):
    numbers = mapl(int, list(data))
    key_map = {}
    
    start = Node(value=numbers[0])
    key_map[start.value] = start
    current = start
    # build a list of nodes
    for i in range(1, len(numbers)):
        node = Node(value=numbers[i])
        key_map[node.value] = node
        current.insert_after(node)
        current = current.cw
    
    return start, key_map

In [70]:
sample_data = "389125467"

In [71]:
def apply_round(current, key_map, max_value=9):
    next_3 = current.remove_next_n(3)
    values = set(next_3.take(3))
    destination = current.find_next_lowest_value(key_map, values, max_value)
    
    destination.insert_after(next_3)
    return current.cw
    

In [86]:
current, key_map = parse_data(data)
for i in range(1, 101):
    # print(f"{i}: {current.to_string()}")
    current = apply_round(current, key_map, max_value=9)
    

one = key_map[1]
result = one.take(9)[1:]
result = "".join(mapl(str, result))

In [87]:
result

'98645732'

In [85]:
submit(result, day=23, part='a')

You don't seem to be solving the right level.  Did you already complete it? [Return to Day 23]


<Response [200]>

# Part 2

In [88]:
from tqdm import tqdm, trange

In [91]:
ROUNDS = 10_000_000
MAX_VALUE = 1_000_000

In [122]:
# Build up giant list
current, key_map = parse_data(data)
for i in trange(10, MAX_VALUE + 1):
    node = Node(i)
    key_map[i] = node
    current.ccw.insert_after(node)


100%|██████████| 999991/999991 [00:08<00:00, 114469.77it/s]


In [123]:
for i in trange(ROUNDS):
    # print(f"{i}: {current.to_string()}")
    current = apply_round(current, key_map, max_value=MAX_VALUE)

one = key_map[1]

100%|██████████| 10000000/10000000 [00:52<00:00, 191857.57it/s]


In [124]:
one.take(3)

[1, 929588, 741727]

In [125]:
import numpy as np

In [126]:
result = np.product(one.take(3)[1:])

In [127]:
result

689500518476

In [128]:
submit(result, day=23)

answer a: 98645732
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 23! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>

It's impossible to run the full simulation?

Just need to figure out where `[1]` ends up.

We know where 1 ends up after the first 10 are processed.

The other 1M being in order means what happens to them should be predictable.

Then can figure out what happens at the end of the list, what gets pushed over into the first 10?

#### observations

- We advance 4 positions through the sequence each time (next 3 and then +1 spot)
- Since 1 is the lowest number, it's the tipping point where we wrap to the 1M number
  - We insert infront of 1 when 1M is current
  - What gets inserted infront of 1 each time? Every 4th number?


In [193]:

# current = parse_data(data)
current = parse_data("389125467")
# current = Node(1)
for i in trange(10, 21):
    current.ccw.insert_after(Node(value=i))


100%|██████████| 11/11 [00:00<00:00, 52849.19it/s]


In [194]:
print(current.to_string())

3, 8, 9, 1, 2, 5, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20


In [ ]:
for i in range(40):
    print(current.to_string())
    print(f"{i}")
    current = apply_round(current, max_value=20)


In [196]:
(10 + 4 * 10_000_000) % 1_000_000

10